In [155]:
import pandas as pd
import numpy as np
from nltk.corpus import words

import re, requests, collections

In [156]:
pbdb_stratigraphic_names = requests.get("https://paleobiodb.org/data1.2/occs/strata.json?datainfo&rowcount&interval=Phanerozoic").json()

l = []
for x in pbdb_stratigraphic_names["records"]:
    if "smb" in x.keys():
        l.append(x["smb"])

In [157]:
# This "interval" "definition" is terrible.
#r = requests.get("https://macrostrat.org/api/defs/intervals?all").json()
#time_intervals = [x["name"].replace("\t", "") for x in r["success"]["data"]]
r = requests.get("https://macrostrat.org/api/defs/strat_names?all").json()
#r = requests.get("https://macrostrat.org/api/defs/strat_names?rank=Fm").json()

In [158]:
strat_names = [x["strat_name_long"] for x in r["success"]["data"]]
#print(len(r["success"]["data"]))
collections.Counter([x.split(" ")[-1] for x in strat_names]).most_common()

[('Formation', 13823),
 ('Member', 9616),
 ('Bed', 3411),
 ('Group', 2356),
 ('Granite', 1851),
 ('Limestone', 1661),
 ('Sandstone', 1426),
 ('Shale', 829),
 ('Granodiorite', 799),
 ('Volcanics', 682),
 ('Complex', 645),
 ('Suite', 629),
 ('Conglomerate', 484),
 ('Gneiss', 439),
 ('Basalt', 431),
 ('Dolomite', 348),
 ('Quartzite', 339),
 ('Rhyolite', 301),
 ('Tuff', 296),
 ('Subgroup', 255),
 ('Schist', 244),
 ('Sand', 208),
 ('Monzogranite', 199),
 ('Tonalite', 195),
 ('Diorite', 191),
 ('Till', 190),
 ('Gravel', 176),
 ('Tongue', 173),
 ('Siltstone', 159),
 ('Supersuite', 159),
 ('Andesite', 150),
 ('Beds', 149),
 ('Gabbro', 148),
 ('Mudstone', 139),
 ('Clay', 127),
 ('Monzonite', 126),
 ('Slate', 117),
 ('Breccia', 111),
 ('Metamorphics', 110),
 ('Drift', 95),
 ('Supergroup', 93),
 ('Ignimbrite', 86),
 ('Measures', 81),
 ('Volcanic', 80),
 ('Dolerite', 75),
 ('Lentil', 65),
 ('Dacite', 64),
 ('Chert', 62),
 ('Dolostone', 60),
 ('Latite', 60),
 ('Phyllite', 59),
 ('Silt', 54),
 ('Mar

In [160]:
with open("data/species_names_bryozoa.txt", "r") as myfile:
	taxa = myfile.read().split("\n")

taxa2 = pd.read_csv("data/CoL_bryozoa.csv")
taxa2 = [' '.join(x.split('  ')[0:2]) for x in list(taxa2["Scientific name"])]
taxa3 = taxa + taxa2

## Problem: some species are written in a bad format, e.g.
## 
## Chaperiopsis (Clipeochaperia) frontalis
##
# ambiguous = [x for x in taxa3 if "(" in x]

In [161]:
philbock_taxa = pd.read_csv("data/philbock_species.csv", sep = "\t", dtype=str)

taxa = [x.capitalize() for x in philbock_taxa.Name.dropna()]
taxa = [x for x in taxa if "subsequent" not in x]

def is_ascii(s):
    return all(ord(c) < 128 for c in s)

#[x for x in philbocktaxa2 if ")" in x]
#[x for x in philbocktaxa2 if "?" in x]
#[x for x in philbocktaxa2 if not is_ascii(x)]
#[x for x in philbocktaxa2 if len(x.split(' ')) > 2][2000:]
#[x.replace(" [sic]", "") for x in taxa]
#[x for x in taxa if "bouei minute" in x]

#re.search("No subsequent", taxa)
[x for x in taxa if " strata" in x]


['Fenestella strata']

In [162]:
def cleanup(genera):
    """
    Function to remove any of the matches in "flags2" from the string input
    """
    flags2 = {"(?)", "*", " [sic]", "?", " cf.", " n. sp.", " 1", 
              " l.", " s.", " j.", 
              " sp.", " aff.", " ex gr.", " [1938]", 
              "\xa0", "\t", " [sic, read costulatus]", " [diastopora]", '&', 
              ' f.', ' l.', ' c.', ' s.',' ex.', 'a.', 'b.', 'grp.',
              ' genus', ' indet.', ' egenus'}
    rep = {x: "" for x in flags2}
    rep = dict((re.escape(k), v) for k, v in rep.items())

    pattern = re.compile("|".join(rep.keys()))
    res = map(lambda genus: pattern.sub(lambda m: rep[re.escape(m.group(0))], genus), (x for x in genera if len(x) > 1))
    return(res)

philbocktaxa = []

taxa = cleanup(taxa)

for taxon in taxa:
    if "sp." in taxon:
        taxon = taxon[:pat.search(taxon).end()]
    
    split_taxon = taxon.split()   
    
    pindices = [i for i, p in enumerate(taxon) if p in "()"]

    if len(pindices) > 1:
        if " " in taxon[pindices[0]:pindices[1]+1]:
            taxon = taxon[:pindices[0]] + taxon[pindices[1]+2:]
            
            taxon = " ".join(taxon.split()) ## Normalize spacing
            philbocktaxa.append(taxon)
            
        else:
            first = taxon.replace(taxon[pindices[0]:pindices[1]+2], "")
            second = taxon[pindices[0]+1:].replace(")", "")
        
            philbocktaxa.append(first.capitalize())
            philbocktaxa.append(second.capitalize())  
        
    
    else:
        taxon = " ".join(taxon.split()) ## Normalize spacing
        philbocktaxa.append(taxon)
        
def nonascii_fix(x):
    x = x.replace("Macropora ﬁlifera", "Macropora filifera")
    x = x.replace("Ralﬁna originalis", "Ralfina originalis")
    x = x.replace("‘hastingsia’ whitteni", "hastingsia whitteni")
    return(x.replace("Biflustra’ lamellosa", "Biflustra lamellosa"))

philbocktaxa2 = [nonascii_fix(x).capitalize() for x in philbocktaxa]
        

philbocktaxa3 = []

for x in philbocktaxa2:
    philbocktaxa3.append(x)
    
    k = x.split()
    if len(k) > 2:
        philbocktaxa3.append(' '.join(k[0:2]))
        

taxa4 = set(x for x in set(taxa3 + philbocktaxa3) if x not in ["Sic", "Delete", "Ignore", "More work", "Zzcomment"])

k = sorted([x.split()[-1] for x in taxa4 if " " in x], key = len)[0:1000]

In [58]:
## Dictionary for english (US) words
#nltk.download("words")
import nltk

english_words = set(words.words())

In [165]:
genera = {x.split(" ")[0] for x in taxa4}
print("Number of genera:", len(genera))

english_genera = {x for x in genera if x in english_words}

print(english_genera)

Number of genera: 3121
{'Stenophragma', 'Mastigophora', 'Favosites', 'Oculina', 'Fungia', 'Isis', 'Monticulipora', 'Stromatopora', 'Paludicella', 'Paulina', 'Lepralia', 'Flustra', 'Hydractinia', 'Tubularia', 'Vesicularia', 'Membranipora', 'Fedora', 'Chaetetes', 'Heliolites', 'Plumatella', 'Cristatella', 'Emma', 'Lunularia', 'Sertularia', 'Lunulites', 'Millepora', 'Alveolites', 'Tubulipora', 'Tubipora', 'Orbitolites', 'Electra', 'Acropora', 'Fistularia', 'Avicularia', 'Retepora', 'Adeona', 'Cellepora', 'Dictyonema', 'Madrepora', 'Hydra', 'Corallina', 'Porella', 'Ulva', 'Campanularia', 'Archimedes', 'Lichenopora', 'Arachnoidea', 'Oncidium', 'Boreas', 'Crepis', 'Gargantua', 'Alcyonium', 'Chaetites', 'Diplotaxis', 'Gorgonia', 'Galeopsis', 'Salicornia'}


In [166]:
# Pick some of the above genera, remove them
bad_genera = {"Emma", "Hydra", "Isis", "Ulva", "Avicularia", "Boreas"}

genera_minus_english = genera.difference(bad_genera)
print("Number of genera, minus english words:", len(genera_minus_english))

with open("data/genera.csv", "w") as f:
    f.write("\n".join(genera_minus_english))

Number of genera, minus english words: 3115


In [167]:
## If the epithet is in the english dictionary, add to set
english_epithets = {x.split()[-1] for x in taxa4 if x.split()[-1] in english_words}

## Geochronological time intervals

First from http://www.stratigraphy.org/bak/geowhen/geolist.html
retrieved 26 march 2018

In [168]:
bad = {"Early", "Late", "Lower", "Middle", "Upper", "III", "II", "VI", "V", "I", "(475", "Stages", "Displayed"}

df = pd.read_csv("data/geolist.txt", sep = "\t", header = None)
k = [x.split(" ")[3:5] for x in df[0]]
stages = [[m for m in x if m not in bad] for x in k]
stages = [[m for m in x if m] for x in stages if x]
stages = {' '.join(x) for x in stages}

In [169]:
with open("data/names_geological.txt", "r") as myfile:
    intervals = myfile.read().split("\n")
if intervals[-1] == "":
    intervals.pop()
        
intervals = set(intervals).union(stages)#.union(manual)

In [170]:
bad_intervals = {"Gulf", "Black", "Early", "Late", "Older", "Oldest", "Arnold", "Canadian", "Cryptic", "Glacial", "Mayan", "Namibian", "Younger",
                        "Primary", "Primordial", "Mayan", "Gallic", "Namibian", "Romanian", "Secondary", 
                 "Sicilian", "Warsaw", "Atokan/Derryan", "Warendian/Lancefieldian",
                #'Donau-Günz', 'Würm', 'Günz-Mindel', 'Allerød', 'Bølling', 'Brüggenian',
                # 'Biber-Donau', 'Pre-Illinoian', 'Nemakit-Daldynian', 'Mindel-Riss',
                # 'Günz', 'Belvédère', 'Riss-Würm'
                }

good_intervals = [x for x in intervals if x not in bad_intervals]

In [183]:
set1 = {x for x in good_intervals if x[-2:] == "an"}
set1de = [x[:-2] + "um" for x in set1]

set2 = [x.replace("gene", "zän") for x in good_intervals if x not in set1]
set3 = [x.replace("cene", "zän") for x in good_intervals if x not in set1]
set4 = [x.replace("ian", "ien") for x in good_intervals]

with open("output/german_names_10_10_2018.txt", "r") as f:
    germanwiki_intervals = f.read().split("\n")
    
german_intervals = set(germanwiki_intervals + set1de + set2 + set3 + set4)

len(german_intervals)

1741

In [193]:
# Strings (or tokens) of species epithets that are not to be added as regexpattern 
flags = {"avicularia", "ta", "moe", "ab", "to", "mui", "lui", 
              'sa', "de", "a", "b", "gay", "genus", "comes",
              "inferior", "biota", "bella", "sinus", "No", "rostrum", "work",
              "major", "minor", "minimum", "fertile", "More", "No", "ordo", "index", "More", "more",
              "Was", "st.", "strata", "foraminifera", "Was", "nuda", "canui", 
              "richteri", "delta", "gigas", "foramina", "fossula", "minima", "americana", "voigti"}
#flags = flags#.union(english_epithets)


override = "ORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME"

In [194]:
#[True for x in rules if "Kreide" in x]

[True]

# Rules 2.0 - with tokensregex


This entry is based on the TokenRegex engine, see

https://nlp.stanford.edu/software/tokensregex.html

```
@techreport{tokensregex2014,
  author    = {Chang, Angel X. and Manning, Christopher D.},
  title  = {{TokensRegex}: Defining cascaded regular expressions over tokens},
  number = {CSTR 2014-02},
  institution = {Department of Computer Science, Stanford University},
  year = {2014}
}
```

In [196]:
genera2 = set()
epithets2 = set()

for species in taxa4:
    k = species.split(" ")
    
    if k[0] not in flags:
        genera2.add(k[0])
    
    if len(k) > 1:
        if k[1] not in flags:
            epithets2.add(k[1])
            
rules = []

rules.append("( [/{}/] [/cf.|aff./]{{0,1}} [/{}/]{{0,1}} [/sp./]{{0,1}} [/nov./]{{0,1}} )\tTAXA\t{}\t2.0".format('|'.join(genera2), 
                                                                                                   '|'.join(epithets2),
                                                                                                  override))

rules.append("( [{{ word:/[A-Z][.]/}}] [/cf.|aff./]{{0,1}} [/{}/] [/sp./]{{0,1}} [/nov./]{{0,1}} )\tTAXA\t{}\t2.0".format('|'.join(epithets2),
                                                                                                     override))

prefixes = {"Late", "late", "Early", "early", "Middle", "middle", "upper", "Upper", "Lower", "lower"}
prefixes_de = {"Oberes?", "oberes?",
             "Niederes?", "nieders?",
             "Spätes?", "spätes?",
             "Frühes?", "frühes?",
             "Mittlere?s?","mittlere?s?",
             "Mittel", "mittel",
             "Unteres?", "unteres?"
            }


rules.append("( [/{}/]{{0,1}} [/{}/] )\tINTERVALNAME\t{}\t2.0".format('|'.join(prefixes),
                                                                       '|'.join(good_intervals),
                                                                       override))

rules.append("( [/{}/]{{0,1}} [/{}/] )\tINTERVALNAME\t{}\t2.0".format('|'.join(prefixes_de),
                                                                       '|'.join(german_intervals),
                                                                       override))

# recent, the recent
rules.append("( [/The|the/]{{0,1}} [/Recent/] )\tINTERVALNAME\t{}\t2.0".format(override))

strat_flags = ["Basalt","basalt","Quartzite","quartzite","Rhyolite","rhyolite","Monzogranite","monzogranite",
               "Gabbro", "gabbro", "Slate", "Breccia", "Ignimbrite", "Volcanic", "Dolerite", "Dacite", "Latite",
               "Phyllite", "Silt", "Marl", "Syenite", "Chalk", "Marble", "Amphibolite", "Greenstone",
               "Orthogneiss", "Tillite", "Leucogranite", "Trachyte", "Argillite", "Arkose", "Greensand",
               "Serpentinite", "Agglomerate", "Pelite", "Rhyodacite", "Greywacke", "Claystone", "Lens", "Ash",
               "Oolite", "Anhydrite", "Graywacke", "Alluvium", "Diabase", "Trondhjemite", "Metagabbro", "Anorthosite",
               "Granulite", "Metabasalt", "Norite", "Evaporite", "Gypsum", "Syenogranite", "Migmatite", "Ironstone",
               "Charnockite", "Peat", "Felsite", "Coquina", "Halite",
               "Siltstone","siltstone","Andesite","andesite","Gneiss","gneiss","Dolomite","dolomite","Dolostone","dolostone",
               "Limestone","limestone","Sandstone","sandstone","Shale","shale","Granite","granite","Granodiorite","granodiorite",
               "Conglomerate","chert","Chert","Group","Formation","formation","Member","Supergroup","supergroup","Bed","Subgroup",
               "subgroup","Fm","Mbr","Gp","SGp"]


stratigraphy = "( [{{ word:/[A-Z][a-z]+/ }}]+? /{}/ )".format('|'.join(strat_flags))

strat_rule = "\t".join([stratigraphy, "STRAT", override, "2.0"])

rules.append(strat_rule)

# Save it

In [197]:
## Save RegexNER rules to file
#with open('output/regexner_rules_15sept.txt', mode='w', encoding='utf-8') as myfile:
with open('/uio/kant/nhm-sfs-u2/bjorntko/natural_language_processing/regexner_rules_9oct.txt', mode='w', encoding='utf-8') as myfile:
    myfile.write('\n'.join(line for line in rules))

## Some summary

By inspecting the file "ner-training/regexner_rules.txt" we can inspect all of the regex expressions, including their target labels and label priorities (1.0 or 2.0). Here, "TAXA" is used for taxa, and "INTERVALNAME" is used for geological time intervals. We can inspect the first entries:

In [188]:
#!head output/regexner_rules_15sept.txt
!echo ""
!wc /uio/kant/nhm-sfs-u2/bjorntko/natural_language_processing/regexner_rules_9oct.txt


     7    103 335733 /uio/kant/nhm-sfs-u2/bjorntko/natural_language_processing/regexner_rules_9oct.txt


In [68]:
'\n'.join(line for line in rules)[-100:]

'subgroup|Fm|Mbr|Gp|SGp/ )\tSTRAT\tORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME\t2.0'